In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from itertools import cycle

# Matplotlib in‑notebook
%matplotlib inline

In [ ]:
# -------------------------------------------------------------
# 0.  Imports
# -------------------------------------------------------------
import pandas as pd
import matplotlib.pyplot as plt
from itertools import cycle

csv_path = "../benchmarks/perf/data/mv_random_perf_full.csv"

# -------------------------------------------------------------
# 1.  Load data, coerce numerics, compute miss %
# -------------------------------------------------------------
num_cols = [
    "time_serial", "time_vectorized", "speedup_time",
    "cycles_serial", "cycles_vector", "speedup_cycles",
    "inst_serial", "inst_vector", "speedup_inst",
    "ipc_serial", "ipc_vector",
    "L1-loads", "L1-misses",
    "LLC-loads", "LLC-misses",
]

df = pd.read_csv(csv_path)
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")

df = (df
      .assign(L1_miss_pct  = lambda d: 100 * d["L1-misses"]  / d["L1-loads"],
              LLC_miss_pct = lambda d: 100 * d["LLC-misses"] / d["LLC-loads"])
      .sort_values(["sparsity", "n"])
      .reset_index(drop=True))

# -------------------------------------------------------------
# 2.  Generic plotting helper
# -------------------------------------------------------------
def plot_speedup_and_miss(data, what="both", title_suffix=""):
    """
    what : 'both' | 'L1' | 'LLC'
    """
    fig, ax1 = plt.subplots(figsize=(8, 5))
    ax2 = ax1.twinx()

    # ---- speed-up (left axis) ---------------------------------
    ax1.set_xlabel("Vector size n")
    ax1.set_xscale("log", base=2)
    ax1.set_ylabel("Speed-up (scalar / vector)")
    ax1.axhline(1.0, color="grey", ls="--", lw=1)
    ax1.axhline(2.0, color="red",  ls="--", lw=1)
    ax1.text(data["n"].max(), 2.03, "2× ideal", color="red",
             ha="right", va="bottom", fontsize=8)

    # ---- miss-rate (right axis) -------------------------------
    ax2.set_ylabel("Cache-miss rate [%]")
    ymax = {
        "L1" :  data["L1_miss_pct"].max(),
        "LLC":  data["LLC_miss_pct"].max(),
        "both": max(data["L1_miss_pct"].max(),
                    data["LLC_miss_pct"].max())
    }[what]
    ax2.set_ylim(0, ymax * 1.05 if ymax > 0 else 1)

    palette = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])

    for spars, g in data.groupby("sparsity"):
        g = g.sort_values("n")
        col = next(palette)

        # speed-up
        ax1.plot(g["n"], g["speedup_cycles"],
                 marker="o", color=col,
                 label=f"Speed-up  s={spars:.2f}")

        # miss % (one or both)
        if what in ("both", "L1"):
            ax2.plot(g["n"], g["L1_miss_pct"],
                     marker="x", ls="--", color=col,
                     label=f"L1 miss %  s={spars:.2f}")
        if what in ("both", "LLC"):
            ax2.plot(g["n"], g["LLC_miss_pct"],
                     marker="s", ls=":", color=col,
                     label=f"LLC miss % s={spars:.2f}")

    # legenda combinata
    h1, l1 = ax1.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()
    ax1.legend(h1 + h2, l1 + l2, loc="best", fontsize=8)

    plt.title(f"Random ELL × v – speed-up & miss {title_suffix}")
    plt.tight_layout()
    plt.show()

# -------------------------------------------------------------
# 3.  Figure globale
# -------------------------------------------------------------
plot_speedup_and_miss(df, what="both", title_suffix="(L1 + LLC)")

# -------------------------------------------------------------
# 4.  Focus su sparsity = 0.01 (facoltativo)
# -------------------------------------------------------------
# df_s001 = df.query("abs(sparsity - 0.01) < 1e-6")
# plot_speedup_and_miss(df_s001, "both", title_suffix="s=0.01")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# ------------------------------------------------------------------
# 1.  Carica e prepara i dati  (↙️ aggiorna il path se serve)
# ------------------------------------------------------------------
csv_file = "../benchmarks/perf/data/mv_foam_perf_full.csv"
df = (pd.read_csv(csv_file)
        .sort_values("n")
        .assign(
            L1_miss_pct  = lambda d: 100 * d["L1-misses"]  / d["L1-loads"],
            LLC_miss_pct = lambda d: 100 * d["LLC-misses"] / d["LLC-loads"],
            L2_miss_pct  = lambda d: 100 * d["L2-misses"]  / d["L2-loads"]
        ))

# ------------------------------------------------------------------
# 2.  Funzione di plotting parametrico
# ------------------------------------------------------------------
def plot_speedup_and_miss(df, *, miss_levels=('L1','LLC','L2'),
                          title_suffix=''):
    """
    Disegna lo speed-up (asse sinistro) e la % di miss cache (asse destro).

    Parameters
    ----------
    miss_levels : iterable {'L1', 'LLC', 'L2'}
        Quali curve di miss-rate disegnare.
        Esempi: ('L1','LLC','L2')   tutte
                 ('L1',)            solo L1
                 ('L2',)            solo L2
    """
    fig, ax1 = plt.subplots(figsize=(8, 5))
    ax2 = ax1.twinx()

    # -- Speed-up ---------------------------------------------------------------
    ax1.set_xlabel("Matrix size n")
    ax1.set_ylabel("Speed-up (cycles scalar / vector)")
    ax1.plot(df["n"], df["speedup_cycles"], marker="o",
             label="Speed-up (cycles)", color="steelblue")
    ax1.axhline(1.0, color="grey", ls="--", lw=1)
    ax1.grid(True, ls=":", which="both")

    # -- Miss-rate --------------------------------------------------------------
    ax2.set_ylabel("Cache miss-rate [%]")

    colors = {'L1': 'forestgreen', 'LLC': 'darkorange', 'L2': 'crimson'}
    styles = {'L1': '--', 'LLC': ':', 'L2': '-.'}
    markers = {'L1': 'x', 'LLC': 's', 'L2': 'd'}

    max_pct = 0
    for level in miss_levels:
        pct_col = f"{level}_miss_pct"
        ax2.plot(df["n"], df[pct_col],
                 marker=markers[level], ls=styles[level],
                 label=f"{level} miss %", color=colors[level])
        max_pct = max(max_pct, df[pct_col].max())

    ax2.set_ylim(0, max_pct * 1.10)

    # -- Legenda combinata ------------------------------------------------------
    h1, l1 = ax1.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()
    ax1.legend(h1 + h2, l1 + l2, loc="best", fontsize=8)

    ax1.axhline(1.0, color="gray", ls="--", lw=1)
    ax1.axhline(2.0, color="red",  ls="--", lw=1)
    ax1.text(df["n"].max(), 2.03, "2× ideal", color="red",
             ha="right", va="bottom", fontsize=8)

    plt.title(f"OpenFOAM MV – speed-up vs miss-rate {title_suffix}")
    plt.tight_layout()
    plt.show()

# ------------------------------------------------------------------
# 3.  Genera le figure richieste
# ------------------------------------------------------------------
plot_speedup_and_miss(df,
    miss_levels=('L1','LLC'),
    title_suffix="(L1 + LLC)")

# plot_speedup_and_miss(df,
#     miss_levels=('L1','LLC','L2'),
#     title_suffix="(tutti i livelli)")

# plot_speedup_and_miss(df,
#     miss_levels=('L1',),
#     title_suffix="(solo L1)")

# plot_speedup_and_miss(df,
#     miss_levels=('LLC',),
#     title_suffix="(solo LLC)")

# plot_speedup_and_miss(df,
#     miss_levels=('L2',),
#     title_suffix="(solo L2)")


In [ ]:
#!/usr/bin/env python3
"""
Plot AXPY speed-up (scalar / vector) + cache-miss rate per L1, LLC, L2.

‣ Il CSV deve avere le colonne:
    n, speedup_cycles,
    L1-loads, L1-misses,
    LLC-loads, LLC-misses,
    L2-loads, L2-misses
‣ L’asse destro (miss-rate) si adatta automaticamente:
    ylim = 0 … 110 % del massimo fra tutte le miss-rate.
"""
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# ------------------------------------------------------------------
# 1) carica i dati  (adatta il path se serve)
# ------------------------------------------------------------------
csv_file = Path("../benchmarks/perf/data/axpy_perf_full.csv")
df = pd.read_csv(csv_file).sort_values("n")

# calcola miss-rate (0–1) per ogni livello
df["L1_miss_rate"]  = df["L1-misses"]  / df["L1-loads"]
df["LLC_miss_rate"] = df["LLC-misses"] / df["LLC-loads"]
# df["L2_miss_rate"]  = df["L2-misses"]  / df["L2-loads"]

# ------------------------------------------------------------------
# 2) plotting
# ------------------------------------------------------------------
fig, ax1 = plt.subplots(figsize=(9, 5))

# ── asse sinistro: speed-up ───────────────────────────────────────
ax1.set_xlabel("Vector size n")
ax1.set_xscale("log", base=2)
ax1.set_ylabel("Speed-up (scalar / vector)")

ax1.plot(df["n"], df["speedup_cycles"],
         marker="o", label="Speed-up (cycles)")

ax1.axhline(1.0, color="gray", linestyle="--", linewidth=1, label="_nolegend_")
ax1.axhline(2.0, color="red",  linestyle="--", linewidth=1, label="_nolegend_")
ax1.text(df["n"].iloc[-1], 2.03, "2× ideal", color="red",
         ha="right", va="bottom", fontsize=8)

ax1.grid(axis="x", which="both", ls=":")

# ── asse destro: miss-rate ───────────────────────────────────────
ax2 = ax1.twinx()
ax2.set_ylabel("Miss rate")

# scala verticale adattiva (+10 % margine)
# miss_max = df[["L1_miss_rate", "LLC_miss_rate", "L2_miss_rate"]].max().max()
miss_max = df[["L1_miss_rate", "LLC_miss_rate"]].max().max()
ax2.set_ylim(0, miss_max * 1.10)
ax2.yaxis.set_major_formatter(lambda x, _: f"{x:.0%}")

# curve miss-rate (si usano i colori default di Matplotlib)
ax2.plot(df["n"], df["L1_miss_rate"],
         marker="s", linestyle="--", label="L1 miss rate")
ax2.plot(df["n"], df["LLC_miss_rate"],
         marker="^", linestyle=":", label="LLC miss rate")
# ax2.plot(df["n"], df["L2_miss_rate"],
#          marker="x", linestyle="-.", label="L2 miss rate")

# ── legenda combinata (ignora label vuote / _nolegend_) ──────────
handles, labels = [], []
for line in ax1.lines + ax2.lines:
    lab = line.get_label()
    if lab and not lab.startswith("_"):
        handles.append(line); labels.append(lab)
ax1.legend(handles, labels, loc="upper right")

plt.title("AXPY – speed-up and cache miss rates (VLSET optimised)")
plt.tight_layout()
plt.show()
